In [1]:
import numpy as np
import pandas as pd
import itertools
import math
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, auc
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
df = pd.read_csv('train.csv', header = 0)
prom_49 = pd.read_csv('promotion_schedule.csv', header = 0)
price_49 = pd.read_csv('price_49.csv', header = 0)
print(df.shape)

(76502, 5)


In [33]:
y = np.ones(df.shape[0])
df['target'] = y

In [38]:
i = [n for n in range(0, 2000)]
j = [n for n in range(0, 40)]
t = [n for n in range(0, 49)]

In [39]:
%%time
X = pd.DataFrame(list(itertools.product(i, j, t)), columns = ['i', 'j', 't'])

Wall time: 2.9 s


In [42]:
price = df.drop_duplicates(subset = ['j','t'])
price.drop(['i', 'target'], axis = 1, inplace = True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [44]:
X = pd.merge(X, price, how='left', on = ['j', 't'])
X.shape

(3920000, 5)

In [45]:
X = pd.merge(X, df[['i', 'j', 't', 'target']], how = 'left', on = ['i', 'j', 't'])
X.shape

(3920000, 6)

In [47]:
X['target'].fillna(0, inplace = True)

In [49]:
X[X['target'] == 1].shape

(76502, 6)

In [51]:
%%time
index = X['price'].index[X['price'].apply(np.isnan)]
for i in index:
    j = int(X.iloc[i]['j'])
    t = int(X.iloc[i]['t'])
    pr = price.loc[(price['j'] == j) & (price['t'] == t-1)]
    if pr['advertised'].iloc[0] == 0: 
        X['price'].iloc[i] = pr['price'].iloc[0]
    else: 
        p = price['price'].loc[(price['j'] == j) & (price['t'] == t-2)]
        X['price'].iloc[i] = p.iloc[0]

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Wall time: 7min 56s


In [17]:
X['advertised'].fillna(0, inplace = True)

In [50]:
#X.to_csv('X.csv')
#X = pd.read_csv('X.csv', header = 0)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [16]:
y = X['target']
X.drop(['target', 't'], axis= 1, inplace = True)

In [ ]:
%%time
#model = LogisticRegressionCV(cv = 3, scoring='auc').fit(X.drop('t', axis = 1), y)
model = XGBClassifier(eval_metric = 'auc').fit(X_train.drop('t', axis = 1), y_train)

In [ ]:
%%time
kf = KFold(n_splits = 2, random_state == 17, shuffle = True)

for train_index, test_index in kf.split(train):
    train_X, valid_X = X[train_index], X[test_index]
    train_y, valid_y = y[train_index], y[test_index]
    
    model = XGBClassifier(eval_metric = 'auc').fit(train_X, train_y)
    xgb_pred = model.predict(valid_X)
    print (auc(xgb_pred, valid_y))

In [22]:
%%time
i = [n for n in range(0, 2000)]
j = [n for n in range(0, 40)]
X_answ = pd.DataFrame(list(itertools.product(i, j)), columns = ['i', 'j'])
X_answ = pd.merge(X_answ, price_49, how = 'left', on = ['j'])
X_asnw = pd.merge(X_answ, prom_49[['j', 'advertised']], how = 'left', on = ['j'])

Wall time: 115 ms


In [31]:
print (X.shape)
print (X.shape)

(3920000, 5)
(80000, 4)


In [302]:
#print (logit.scores_)

{1.0: array([[ 0.98048219,  0.98048219,  0.98048219,  0.98048219,  0.98048219,
         0.98048219,  0.98048219,  0.98048219,  0.98048219,  0.98048219],
       [ 0.98048219,  0.98048219,  0.98048219,  0.98048219,  0.98048219,
         0.98048219,  0.98048219,  0.98048219,  0.98048219,  0.98048219],
       [ 0.98048469,  0.98048469,  0.98048469,  0.98048469,  0.98048469,
         0.98048469,  0.98048469,  0.98048469,  0.98048469,  0.98048469],
       [ 0.98048469,  0.98048469,  0.98048469,  0.98048469,  0.98048469,
         0.98048469,  0.98048469,  0.98048469,  0.98048469,  0.98048469],
       [ 0.98048469,  0.98048469,  0.98048469,  0.98048469,  0.98048469,
         0.98048469,  0.98048469,  0.98048469,  0.98048469,  0.98048469],
       [ 0.98048469,  0.98048469,  0.98048469,  0.98048469,  0.98048469,
         0.98048469,  0.98048469,  0.98048469,  0.98048469,  0.98048469],
       [ 0.98048469,  0.98048469,  0.98048469,  0.98048469,  0.98048469,
         0.98048469,  0.98048469,  0.98

In [52]:
y_answ = model.predict(X_test)
#y_test = model.predict_proba(X_test)[:, 1]
y_answ = pd.DataFrame(y_answ.reshape(len(y_answ), -1), columns = ['target'])

In [ ]:
X_answ = pd.concat([X_answ, y_answ], axis = 1)
X_answ.drop('discount', axis = 1, inplace = True)

In [307]:
y_answ.loc[y_train == 0].shape

(3843498,)

,target


In [310]:
X_train.head()

,i,j,t,price,advertised
0,0,0,0,1.717944,0.0
1,0,0,1,1.717944,0.0
2,0,0,2,1.717944,0.0
3,0,0,3,1.717944,0.0
4,0,0,4,1.717944,0.0
